전체 구조 : Affine-ReLU => Conv-ReLU-(Pooling)

## 7.2 합성곱 계층
* fully connected의 문제점 : 데이터의 형상이 무시된다
* (책에서의 정의) **feature map** : 입출력 데이터
* **fused multiply-add (FMA)** : feature map과 filter에서 대응하는 원소끼리 곱한 후 그 총합을 구하는 것
  * 이 연산 때문에 3채널에 대응하는 필터 3개를 사용하더라도 output은 1개만 나오는 것이다
* padding은 주로 output size를 조정하려는 목적으로 사용한다   
  e.g. (4,4)에 (3,3) 필터를 패딩없이 적용하면 출력 사이즈가 2 줄어듦 -> 합성곱 연산이 반복되는 DNN에서는 문제가 될 수 있다 (어느 시점엔 출력 사이즈가 1이 되어 합성곱 더 못함)
$$OH=\frac{H+2P-FH}{S}+1 \\ OW=\frac{W+2P-FW}{S}+1$$   
where input size is (H, W), filter size is (FH, FW)
* 이미지 데이터는 3채널이니까 filter 3개로, 하지만 output이 3개가 되진 않는다
  * 하나의 input 덩어리의 각 채널에 대응하는 필터들은 서로 같은 크기여야 함
  * 그렇다면 output은 항상 1개만 내보낼 수 있는가: 아니다, 필터를 다수 사용할 수 있다: (C,H,W)인 한덩이 input에 FN개의 필터(FN,C,H,W)를 적용하면 output map도 FN개가 생긴다(FN, OH, OW)
  * Bias는 (FN, 1, 1)이 되겠다

배치처리
* (N, C, H, W) -> (N, FN, OH, OW)







## 7.3 풀링 계층
* e.g. max pooling, average pooling
  * 이미지 인식 분야에서는 주로 최대 풀링 사용
* pooling 할 때는 window size와 stride를 같게 하는게 보통


## 7.4 합성곱/풀링 계층 구현하기
* Numpy에서는 원소에 접근할 때 for문을 안쓰는게 바람직하다


In [2]:
import numpy as np

In [ ]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [ ]:
x[0, 0].shape

(28, 28)

### 7.4.2 im2col로 데이터 전개하기
* Numpy에서는 원소에 접근할 때 for문을 안쓰는게 바람직하다
* im2col은 입력 데이터를 필터링하기 좋게 전개하는 함수
  * 4차원 입력 데이터에서 하나의 필터가 적용되는 박스를 1줄로
* 보기에 좋게끔 stride를 크게 잡아 필터의 적용 영역이 겹치지 않도록 했지만, 실제 상황에서는 영역이 겹치는 경우가 대부분이다   
  -> im2col로 전개 후 원소 수가 원래 블록의 원소 수보다 많아진다   
  (im2col은 메모리 비효율적인 방식)   
  (컴퓨터는 큰 행렬을 묶어서 계산하는데 탁월하다 - 선형 대수 라이브러리로 큰 행렬의 곱셉을 빠르게!)
* 이제 입력데이터가 행렬이 됐으니, 필터 역시 행렬형태로 만들어서 곱


In [8]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).

    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


### 개인 테스트 - numpy의 transpose, reshape

In [ ]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).

    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    print("img:\n",img)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    print("col:\n",col)
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
    print("after:\n", col)
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


x = np.array([[[[1,2,3],[4,5,6],[7,8,9]],[[-1,-2,-3],[-4,-5,-6],[-7,-8,-9]]],[[[11,12,13],[14,15,16],[17,18,19]],[[-11,-12,-13],[-14,-15,-16],[-17,-18,-19]]]])
a = im2col(x, 2, 2, stride=1, pad=0)
print(a)

img:
 [[[[  1   2   3]
   [  4   5   6]
   [  7   8   9]]

  [[ -1  -2  -3]
   [ -4  -5  -6]
   [ -7  -8  -9]]]


 [[[ 11  12  13]
   [ 14  15  16]
   [ 17  18  19]]

  [[-11 -12 -13]
   [-14 -15 -16]
   [-17 -18 -19]]]]
col:
 [[[[[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]


   [[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]]



  [[[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]


   [[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]]]




 [[[[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]


   [[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]]



  [[[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]


   [[[0. 0.]
     [0. 0.]]

    [[0. 0.]
     [0. 0.]]]]]]
after:
 [[[[[[  1.   2.]
     [  4.   5.]]

    [[  2.   3.]
     [  5.   6.]]]


   [[[  4.   5.]
     [  7.   8.]]

    [[  5.   6.]
     [  8.   9.]]]]



  [[[[ -1.  -2.]
     [ -4.  -5.]]

    [[ -2.  -3.]
     [ -5.  -6.]]]


   [[[ -4.  -5.]
     [ -7.  -8.]]

    [[ -5.  -6.]
     [ -8.  -9.

In [ ]:
x2=np.arange(48).reshape(4,12)
x2

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]])

In [ ]:
x2 = x2.reshape(4,2,6)
x2
# x2.reshape(4,6,2)

array([[[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]],

       [[12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]],

       [[24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35]],

       [[36, 37, 38, 39, 40, 41],
        [42, 43, 44, 45, 46, 47]]])

In [ ]:
x2_trans = x2.transpose(0,2,1)
x2_trans

array([[[ 0,  6],
        [ 1,  7],
        [ 2,  8],
        [ 3,  9],
        [ 4, 10],
        [ 5, 11]],

       [[12, 18],
        [13, 19],
        [14, 20],
        [15, 21],
        [16, 22],
        [17, 23]],

       [[24, 30],
        [25, 31],
        [26, 32],
        [27, 33],
        [28, 34],
        [29, 35]],

       [[36, 42],
        [37, 43],
        [38, 44],
        [39, 45],
        [40, 46],
        [41, 47]]])

In [ ]:
x2_trans.reshape(24,-1)

array([[ 0,  6],
       [ 1,  7],
       [ 2,  8],
       [ 3,  9],
       [ 4, 10],
       [ 5, 11],
       [12, 18],
       [13, 19],
       [14, 20],
       [15, 21],
       [16, 22],
       [17, 23],
       [24, 30],
       [25, 31],
       [26, 32],
       [27, 33],
       [28, 34],
       [29, 35],
       [36, 42],
       [37, 43],
       [38, 44],
       [39, 45],
       [40, 46],
       [41, 47]])

### 구현

In [ ]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [ ]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    N, C, H, W = x.shape
    FN, C, FH, FW = self.W.shape
    out_H = 1 + (H + 2*self.pad - FH) / self.stride
    out_W = 1 + (W + 2*self.pad - FW) / self.stride

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T
    out = np.dot(col, col_W) + self.b
    out = out.reshape(N, out_H, out_W, -1).transpose(0, 3, 1, 2)
    return out


In [28]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.

    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [42]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

    self.x = None
    self.col = None
    self.col_W = None

    self.dW = None
    self.db = None

  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
    out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T
    out = np.dot(col, col_W) + self.b
    out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)

    self.x = x
    self.col = col
    self.col_W = col_W

    return out

  # backward 추가
  def backward(self, dout):
    FN, C, FH, FW = self.W.shape
    dout = dout.transpose(0,2,3,1).reshape(-1, FN)

    self.db = np.sum(dout, axis=0)
    self.dW = np.dot(self.col.T, dout)
    # out.reshape 하던거 참조
    self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

    dcol = np.dot(dout, self.col_W.T)
    dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

    return dx

In [29]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad

    self.x = None
    self.arg_max = None

  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)
    print(out_h, out_w)
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)

    arg_max = np.argmax(col, axis=1)
    out = np.max(col, axis=1)
    out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

    self.x = x
    self.arg_max = arg_max

    return out

  def backward(self, dout):
    # C-dim 다시 맨 뒤로 돌리고
    dout = dout.transpose(0,2,3,1)

    pool_size = self.pool_h * self.pool_w
    dmax = np.zeros((dout.size, pool_size))
    dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()

    dmax = dmax.reshape(dout.shape + (pool_size, ))
    # C-dim 뒤쪽 차원으로 병합
    dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
    dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
    return dx

x1 = np.random.rand(3,3,4,4)
dout = np.random.rand(3,3,2,2)
layer = Pooling(2,2,2,0)
layer.forward(x1)
layer.arg_max.flatten()
layer.backward(dout)

2 2
(3, 2, 2, 3)
(3, 2, 2, 3, 4)


array([[[[0.        , 0.        , 0.        , 0.67044796],
         [0.        , 0.52244178, 0.        , 0.        ],
         [0.        , 0.05853519, 0.00914409, 0.        ],
         [0.        , 0.        , 0.        , 0.        ]],

        [[0.21072556, 0.        , 0.23189859, 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.28335701, 0.        , 0.40706139, 0.        ],
         [0.        , 0.        , 0.        , 0.        ]],

        [[0.38333943, 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.05733793, 0.        ],
         [0.        , 0.20664383, 0.24172443, 0.        ],
         [0.        , 0.        , 0.        , 0.        ]]],


       [[[0.        , 0.        , 0.70992835, 0.        ],
         [0.        , 0.20110017, 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.16316316, 0.15891281, 0.        ]],

        [[0.        , 0.        , 0.        , 

## 7.5 CNN 구현하기

In [30]:
from collections import OrderedDict

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


class Relu:
  def __init__(self):
      self.mask = None

  def forward(self, x):
      self.mask = (x <= 0)
      out = x.copy()
      out[self.mask] = 0

      return out

  def backward(self, dout):
      dout[self.mask] = 0
      dx = dout

      return dx

class Affine:
  def __init__(self, W, b):
      self.W = W
      self.b = b

      self.x = None
      self.original_x_shape = None
      # 가중치와 편향 매개변수의 미분
      self.dW = None
      self.db = None

  def forward(self, x):
      # 텐서 대응
      self.original_x_shape = x.shape
      x = x.reshape(x.shape[0], -1)
      self.x = x

      out = np.dot(self.x, self.W) + self.b

      return out

  def backward(self, dout):
      dx = np.dot(dout, self.W.T)
      self.dW = np.dot(self.x.T, dout)
      self.db = np.sum(dout, axis=0)

      dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
      return dx

class SoftmaxWithLoss:
  def __init__(self):
      self.loss = None # 손실함수
      self.y = None    # softmax의 출력
      self.t = None    # 정답 레이블(원-핫 인코딩 형태)

  def forward(self, x, t):
      self.t = t
      self.y = softmax(x)
      self.loss = cross_entropy_error(self.y, self.t)

      return self.loss

  def backward(self, dout=1):
      batch_size = self.t.shape[0]
      if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
          dx = (self.y - self.t) / batch_size
      else:
          dx = self.y.copy()
          dx[np.arange(batch_size), self.t] -= 1
          dx = dx / batch_size

      return dx


def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val # 값 복원
        it.iternext()

    return grad


In [31]:
class SimpleConvNet:
    """단순한 합성곱 신경망

    conv - relu - pool - affine - relu - affine - softmax

    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [40]:
class Adam:

    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None

    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)

        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

        for key in params.keys():
            #self.m[key] = self.beta1*self.m[key] + (1-self.beta1)*grads[key]
            #self.v[key] = self.beta2*self.v[key] + (1-self.beta2)*(grads[key]**2)
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])

            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)

            #unbias_m += (1 - self.beta1) * (grads[key] - self.m[key]) # correct bias
            #unbisa_b += (1 - self.beta2) * (grads[key]*grads[key] - self.v[key]) # correct bias
            #params[key] += self.lr * unbias_m / (np.sqrt(unbisa_b) + 1e-7)

class Trainer:
    """신경망 훈련을 대신 해주는 클래스
    """
    def __init__(self, network, x_train, t_train, x_test, t_test,
                 epochs=20, mini_batch_size=100,
                 optimizer='SGD', optimizer_param={'lr':0.01},
                 evaluate_sample_num_per_epoch=None, verbose=True):
        self.network = network
        self.verbose = verbose
        self.x_train = x_train
        self.t_train = t_train
        self.x_test = x_test
        self.t_test = t_test
        self.epochs = epochs
        self.batch_size = mini_batch_size
        self.evaluate_sample_num_per_epoch = evaluate_sample_num_per_epoch

        # optimzer
        optimizer_class_dict = {'adam':Adam}
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)

        self.train_size = x_train.shape[0]
        self.iter_per_epoch = max(self.train_size / mini_batch_size, 1)
        self.max_iter = int(epochs * self.iter_per_epoch)
        self.current_iter = 0
        self.current_epoch = 0

        self.train_loss_list = []
        self.train_acc_list = []
        self.test_acc_list = []

    def train_step(self):
        batch_mask = np.random.choice(self.train_size, self.batch_size)
        x_batch = self.x_train[batch_mask]
        t_batch = self.t_train[batch_mask]

        grads = self.network.gradient(x_batch, t_batch)
        self.optimizer.update(self.network.params, grads)

        loss = self.network.loss(x_batch, t_batch)
        self.train_loss_list.append(loss)
        if self.verbose: print("train loss:" + str(loss))

        if self.current_iter % self.iter_per_epoch == 0:
            self.current_epoch += 1

            x_train_sample, t_train_sample = self.x_train, self.t_train
            x_test_sample, t_test_sample = self.x_test, self.t_test
            if not self.evaluate_sample_num_per_epoch is None:
                t = self.evaluate_sample_num_per_epoch
                x_train_sample, t_train_sample = self.x_train[:t], self.t_train[:t]
                x_test_sample, t_test_sample = self.x_test[:t], self.t_test[:t]

            train_acc = self.network.accuracy(x_train_sample, t_train_sample)
            test_acc = self.network.accuracy(x_test_sample, t_test_sample)
            self.train_acc_list.append(train_acc)
            self.test_acc_list.append(test_acc)

            if self.verbose: print("=== epoch:" + str(self.current_epoch) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc) + " ===")
        self.current_iter += 1

    def train(self):
        for i in range(self.max_iter):
            self.train_step()

        test_acc = self.network.accuracy(self.x_test, self.t_test)

        if self.verbose:
            print("=============== Final Test Accuracy ===============")
            print("test acc:" + str(test_acc))

In [35]:
from tensorflow.keras.datasets import mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()

x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)


print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 1, 28, 28)
(60000,)
(10000, 1, 28, 28)
(10000,)


In [44]:
import matplotlib.pyplot as plt

# 시간이 오래 걸릴 경우 데이터를 줄인다.
x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

cnn = SimpleConvNet(input_dim=(1,28,28),
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)

trainer = Trainer(cnn, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
# cnn.save_params("params.pkl")
# print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.08294623225337093
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.04778055314581458
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.06642595419201296
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.04811889470022745
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.053347891571154805
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.12243303014339882
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.051361390531105215
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.0958983039165398
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.1185793426040622
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.10992942439805414
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:0.12709767943032882
12 12
(100, 12, 12, 30)
(100, 12, 12, 30, 4)
12 12
train loss:

NameError: name 'pickle' is not defined

## 7.6 CNN 시각화하기
* 필터가 보고 있는 것
  1. edge : 색상이 바뀐 경계선
  2. blob : 국소적으로 덩어리진 영역
* 딥러닝 시각화에 관한 연구에 따르면 conv&pool 계층을 깊게 쌓을수록 추출되는 정보가 더 추상화 된다

## 7.7 대표적인 CNN
LeNet
* sigmoid를 사용하는 데 반해, 현재는 주로 ReLU 사용
* subsampling하는 pooling을 사용하는 데 반해, 현재는 max pooling이 주류    

AlexNet
* ReLU 사용
* Local Response Normalization이라는 국소적 정규화를 실시하는 계층 존재
* dropout 사용
